In [ ]:
import sys
import math
from numbers import Number
import numpy as np
from numpy.testing import assert_allclose
import contextlib

from qiskit import QuantumCircuit
from qiskit.circuit import Gate

from qiskit.quantum_info.random import random_unitary, random_statevector
from qiskit_aer import AerSimulator
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp

from qiskit_aer.quantum_info.states.aer_state import AerState

In [ ]:
def rand_vec(n, normalize=False):
    """Return complex vector or AerStatevector"""
    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)
    vec = rng.random(n) + 1j * rng.random(n)
    if normalize:
        vec /= np.sqrt(np.dot(vec, np.conj(vec)))
    return vec

def rand_rho(n):
        """Return random pure state density matrix"""
        rho = rand_vec(n, normalize=True)
        return np.outer(rho, np.conjugate(rho))

def assertEqual(a, b):
    if isinstance(a, np.ndarray):
        assert np.all(a == b)
    else:
        assert a == b

def assertTrue(a):
    assertEqual(a, True)

def assertFalse(a):
    assertEqual(a, False)

def assertIs(a, b):
    assertTrue(a is b)

def assertIsNot(a, b):
    assertTrue(a is not b)

def assertIsNone(a):
    assertTrue(a is None)

def assertIn(a, b):
    assertTrue(a in b)

def assertRaises(exception, func, *args, **kwds):
    try:
        func(*args, **kwds)
    except exception:
        pass

def assertAlmostEqual(a, b):
    assertTrue(np.isclose(a, b))

# https://stackoverflow.com/questions/23549419/assert-that-two-dictionaries-are-almost-equal
def _assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    """
    If dictionary value is a number, then check that the numbers are almost equal, otherwise check if values are exactly equal
    Note: does not currently try converting strings to digits and comparing them. Does not care about ordering of keys in dictionaries
    Just returns true or false
    """
    if len(dict1) != len(dict2):
        return False
    # Loop through each item in the first dict and compare it to the second dict
    for key, item in dict1.items():
        # If it is a nested dictionary, need to call the function again
        if isinstance(item, dict):
            # If the nested dictionaries are not almost equal, return False
            if not _assertDictAlmostEqual(dict1[key], dict2[key], rel_tol=rel_tol):
                return False
        # If it's not a dictionary, then continue comparing
        # Put in else statement or else the nested dictionary will get compared twice and
        # On the second time will check for exactly equal and will fail
        else:
            # If the value is a number, check if they are approximately equal
            if isinstance(item, Number):
                # if not abs(dict1[key] - dict2[key]) <= rel_tol:
                # https://stackoverflow.com/questions/5595425/what-is-the-best-way-to-compare-floats-for-almost-equality-in-python
                if hasattr(item, 'imag'):
                    if not math.isclose(dict1[key].real, dict2[key].real, rel_tol=rel_tol) or not math.isclose(dict1[key].imag, dict2[key].imag, rel_tol=rel_tol):
                        return False
                else:
                    if not math.isclose(dict1[key], dict2[key], rel_tol=rel_tol):
                        return False
            else:
                if not (dict1[key] == dict2[key]):
                    return False
    return True

def assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    assertTrue(_assertDictAlmostEqual(dict1, dict2, rel_tol))

@contextlib.contextmanager
def subTest(**kwds):
    try:
        yield
    except Exception as e:
        msg = kwds['msg'] if 'msg' in kwds else ''
        print(e, msg, file=sys.stderr)

In [ ]:
"""Test generation of AerState"""
state = AerState()

In [ ]:
"""Test move of aer state to python"""
state = AerState()
state.allocate_qubits(4)
state.initialize()
sv = state.move_to_ndarray()
state.close()
assertEqual(len(sv), 2**4)

In [ ]:
"""Test reuse AerState after move of aer state to python"""
state = AerState()
state.allocate_qubits(4)
state.initialize()
sv = state.move_to_ndarray()

assertRaises(Exception, state.allocate_qubits, 4)

state.close()

In [ ]:
"""Test initialization of AerState with statevector"""
state1 = AerState()
state1.allocate_qubits(4)
state1.initialize()
sv1 = state1.move_to_ndarray()
state1.close()

assertEqual(sv1[0], complex(1., 0.))
for idx in range(1, len(sv1)):
    assertEqual(sv1[idx], complex(0., 0.))

sv1[0] = complex(0., 0.)
sv1[len(sv1) - 1] = complex(1., 0.)

state2 = AerState()
state2.initialize(sv1)
state2.flush()
sv2 = state2.move_to_ndarray()
state2.close()

for idx in range(len(sv2) - 2):
    assertEqual(sv2[idx], complex(0., 0.))
assertEqual(sv2[len(sv2) - 1], complex(1., 0.))

In [ ]:
"""Test initialization of AerState with densitymatrix"""
state1 = AerState(method='density_matrix')
state1.allocate_qubits(4)
state1.initialize()

dm1 = state1.move_to_ndarray()
assertEqual((16, 16), dm1.shape)

for row in range(dm1.shape[0]):
    for col in range(dm1.shape[1]):
        if row == 0 and col == 0:
            assertEqual(dm1[row][col], complex(1., 0.))
        else:
            assertEqual(dm1[row][col], complex(0., 0.))

dm1[0][0] = complex(0., 0.)
dm1[len(dm1) - 1][len(dm1) - 1] = complex(1., 0.)
state1.close()

state2 = AerState(method='density_matrix')
state2.initialize(dm1, False)
state2.flush()
dm2 = state2.move_to_ndarray()
state2.close()

assertEqual((16, 16), dm2.shape)

for row in range(dm2.shape[0]):
    for col in range(dm2.shape[1]):
        if row == len(dm2) - 1 and col == len(dm2) - 1:
            assertEqual(dm2[row][col], complex(1., 0.))
        else:
            assertEqual(dm2[row][col], complex(0., 0.))

In [ ]:
"""Test initialization of AerState with statevector"""
init_state = random_statevector(2**5, seed=111)
state1 = AerState(seed_simulator=2222)
state1.allocate_qubits(4)
state1.initialize(init_state.data, copy=True)
sample1 = state1.sample_counts()
sv1 = state1.move_to_ndarray()

state2 = AerState(seed_simulator=2222)
state2.initialize(sv1, copy=False)
sample2 = state2.sample_counts()
sv2 = state2.move_to_ndarray()
state2.close()

assertIs(sv1, sv2)
assertEqual(sample1, sample2)

In [ ]:
"""Test initialization of AerState with densitymatrix"""
init_state = random_statevector(4**4, seed=111).data.reshape(16, 16)
state1 = AerState(method='density_matrix', seed_simulator=2222)
state1.initialize(init_state, copy=True)
sample1 = state1.sample_counts()
dm1 = state1.move_to_ndarray()

state2 = AerState(method='density_matrix', seed_simulator=2222)
state2.initialize(dm1, copy=False)
sample2 = state2.sample_counts()
dm2 = state2.move_to_ndarray()
state2.close()

assertIs(dm1, dm2)
assertEqual(sample1, sample2)

In [ ]:
"""Test initialization of AerState with statevector"""
state1 = AerState()
state1.allocate_qubits(4)
state1.initialize()
sv1 = state1.move_to_ndarray()
sv1[0] = complex(0., 0.)
sv1[len(sv1) - 1] = complex(1., 0.)
state1.close()

for _ in range(100):
    state2 = AerState()
    state2.initialize(sv1, copy=False)
    sv2 = state2.move_to_ndarray()
    state2.close()

    for idx in range(len(sv2) - 2):
        assertEqual(sv2[idx], complex(0., 0.))
    assertEqual(sv2[len(sv2) - 1], complex(1., 0.))

In [ ]:
"""Test initialization of AerState with densitymatrix"""
state1 = AerState(method='density_matrix')
state1.allocate_qubits(4)
state1.initialize()

dm1 = state1.move_to_ndarray()
dm1[0][0] = complex(0., 0.)
dm1[len(dm1) - 1][len(dm1) - 1] = complex(1., 0.)
state1.close()

for _ in range(100):
    state2 = AerState(method='density_matrix')
    state2.initialize(dm1, copy=False)
    dm2 = state2.move_to_ndarray()
    state2.close()

    for row in range(dm2.shape[0]):
        for col in range(dm2.shape[1]):
            if row == len(dm2) - 1 and col == len(dm2) - 1:
                assertEqual(dm2[row][col], complex(1., 0.))
            else:
                assertEqual(dm2[row][col], complex(0., 0.))

In [ ]:
"""Test initialization of AerState with normal ndarray"""
sv1 = np.zeros((2 ** 4), dtype=np.complex128)
sv1[len(sv1) - 1] = 1.

state1 = AerState()
state1.initialize(sv1)

sv2 = state1.move_to_ndarray()
assertIsNot(sv1, sv2)
assertEqual(len(sv1), len(sv2))
assertEqual(sv1[len(sv1) - 1], sv2[len(sv2) - 1])

state1.close()

In [ ]:
"""Test initialization of AerState with normal ndarray"""
sv1 = np.zeros((2 ** 4), dtype=np.complex128)
sv1[len(sv1) - 1] = 1.

state1 = AerState()
state1.initialize(sv1, copy=False)

sv2 = state1.move_to_ndarray()
assertIs(sv1, sv2)

state1.close()

In [ ]:
"""Test applying a unitary matrix"""
unitary_1 = random_unitary(2, seed=1111)
unitary_2 = random_unitary(4, seed=2222)
unitary_3 = random_unitary(8, seed=3333)

circuit = QuantumCircuit(5)
circuit.unitary(unitary_1, [0])
circuit.unitary(unitary_2, [1, 2])
circuit.unitary(unitary_3, [3, 4, 0])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize()

state.apply_unitary([0], unitary_1)
state.apply_unitary([1, 2], unitary_2)
state.apply_unitary([3, 4, 0], unitary_3)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a multiplexer operation"""
class CustomMultiplexer(Gate):

    def validate_parameter(self, param):
        return param

def multiplexer_multi_controlled_x(num_control):
    identity = np.array(np.array([[1, 0], [0, 1]], dtype=complex))
    x_gate = np.array(np.array([[0, 1], [1, 0]], dtype=complex))
    num_qubits = num_control + 1
    multiplexer = CustomMultiplexer('multiplexer',
            num_qubits, (2 ** num_control-1) * [identity] + [x_gate],
    )
    return multiplexer

multiplexr_1 = multiplexer_multi_controlled_x(1)
multiplexr_2 = multiplexer_multi_controlled_x(2)
multiplexr_3 = multiplexer_multi_controlled_x(3)

init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.append(multiplexr_1, [0, 1])
circuit.append(multiplexr_2, [1, 2, 3])
circuit.append(multiplexr_3, [3, 4, 0, 1])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_multiplexer([1], [0], multiplexr_1.params)
state.apply_multiplexer([2, 3], [1], multiplexr_2.params)
state.apply_multiplexer([4, 0, 1], [3], multiplexr_3.params)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a diagonal gate"""
diag_1 = [1, -1]
diag_2 = [1, -1, -1, 1]
diag_3 = [1, -1, 1, -1, 1, -1, 1, -1]

init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.diagonal(diag_1, [0])
circuit.diagonal(diag_2, [1, 2])
circuit.diagonal(diag_3, [3, 4, 0])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_diagonal([0], diag_1)
state.apply_diagonal([1, 2], diag_2)
state.apply_diagonal([3, 4, 0], diag_3)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a mcx gate"""
class MCX(Gate):

    def validate_parameter(self, param):
        return param

def mcx(num_control):
    return MCX('mcx', num_control + 1, [])

init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.append(mcx(1), [0, 1])
circuit.append(mcx(2), [1, 2, 3])
circuit.append(mcx(3), [4, 0, 1, 2])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_mcx([0], 1)
state.apply_mcx([1, 2], 3)
state.apply_mcx([4, 0, 1], 2)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a mcy gate"""
class MCY(Gate):

    def validate_parameter(self, param):
        return param

def mcy(num_control):
    return MCY('mcy', num_control + 1, [])

init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.append(mcy(1), [0, 1])
circuit.append(mcy(2), [1, 2, 3])
circuit.append(mcy(3), [4, 0, 1, 2])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_mcy([0], 1)
state.apply_mcy([1, 2], 3)
state.apply_mcy([4, 0, 1], 2)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a mcz gate"""
class MCZ(Gate):

    def validate_parameter(self, param):
        return param

def mcz(num_control):
    return MCZ('mcz', num_control + 1, [])

init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.append(mcz(1), [0, 1])
circuit.append(mcz(2), [1, 2, 3])
circuit.append(mcz(3), [4, 0, 1, 2])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_mcz([0], 1)
state.apply_mcz([1, 2], 3)
state.apply_mcz([4, 0, 1], 2)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a rest gate"""
seed = 1234
init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.reset(0)
circuit.reset([2, 4])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector', seed_simulator=seed)
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState(seed_simulator=seed)
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_reset([0])
state.apply_reset([2, 4])
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test applying a measure"""
seed = 1234
init_state = random_statevector(2**5, seed=111)

circuit = QuantumCircuit(5, 1)
circuit.initialize(init_state, [0, 1, 2, 3, 4])
circuit.measure(0, 0)
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector', seed_simulator=seed)
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState(seed_simulator=seed)
state.allocate_qubits(5)
state.initialize(init_state.data)

state.apply_measure([0])
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)

In [ ]:
"""Test probability() of outcome"""
init_state = random_statevector(2**5, seed=111)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

expected = init_state.probabilities()

for idx in range(0, 2**5):
    assertAlmostEqual(state.probability(idx), expected[idx])

In [ ]:
"""Test probabilities() of outcome"""
init_state = random_statevector(2**5, seed=111)

state = AerState()
state.allocate_qubits(5)
state.initialize(init_state.data)

expected = init_state.probabilities()
actual = state.probabilities()

for idx in range(0, 2**5):
    assertAlmostEqual(actual[idx], expected[idx])

In [ ]:
"""Test sampling"""
init_state = random_statevector(2**5, seed=111)

aer_simulator = AerSimulator(method='statevector')
circuit = QuantumCircuit(5)
circuit.initialize(init_state.data)
circuit.measure_all()
result = aer_simulator.run(circuit, seed_simulator=11111).result()
expected = result.get_counts(0)

state = AerState(seed_simulator=11111)
state.allocate_qubits(5)
state.initialize(init_state.data)
actual = state.sample_counts()

for key, value in actual.items():
    key_str = f"{key:05b}"
    expected_val = expected[key_str] if key_str in expected else 0
    assertAlmostEqual(actual[key], expected_val)

In [ ]:
"""Test global phase"""
unitary_1 = random_unitary(2, seed=1111)
unitary_2 = random_unitary(4, seed=2222)
unitary_3 = random_unitary(8, seed=3333)

circuit = QuantumCircuit(5, global_phase=np.pi/4)
circuit.unitary(unitary_1, [0])
circuit.unitary(unitary_2, [1, 2])
circuit.unitary(unitary_3, [3, 4, 0])
circuit.save_statevector()

aer_simulator = AerSimulator(method='statevector')
result = aer_simulator.run(circuit).result()
expected = result.get_statevector(0)

state = AerState()
state.allocate_qubits(5)
state.initialize()

state.apply_global_phase(np.pi/4)

state.apply_unitary([0], unitary_1)
state.apply_unitary([1, 2], unitary_2)
state.apply_unitary([3, 4, 0], unitary_3)
actual = state.move_to_ndarray()

for i, amp in enumerate(actual):
    assertAlmostEqual(expected[i], amp)